In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# -*- coding: utf-8 -*-
import argparse
import math
import sys
import time
import copy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization#, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

#K.set_image_dim_ordering('th')
#print(K.image_data_format())

## required for efficient GPU use
#import tensorflow as tf
#from keras.backend import tensorflow_backend
#config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#session = tf.Session(config=config)
#tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import h5py

Instructions for updating:
non-resource variables are not supported in the long term


In [18]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Shallow CNN Model/results/cnn_shallow_mawi1.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [19]:
# get data

class dataset:
    mawi_train_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl")
    mawi_test_2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(002).pkl")
    
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['status_anomaly','status_normal']
    
    x_input = dataset.mawi_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.mawi_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.mawi_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.mawi_test_2labels.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (40934, 4)
y_train shape: (40934, 2)
x_test shape: (47265, 4)
y_test shape: (47265, 2)


In [20]:
#reshape data
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (40934, 4, 1)
test shape after reshape: (47265, 4, 1)


In [21]:
#size of parameters
batch_size = 64
num_classes = 2
epochs = 10
filter_size=3
#noise = 1
droprate=0.50

In [22]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(4, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=1))

#FCN layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 4, 64)             256       
                                                                 
 batch_normalization_4 (Batc  (None, 4, 64)            256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 4, 64)             0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 3, 64)            0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 192)               0         
                                                

In [23]:
#Save Model=ON
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 40934 samples, validate on 47265 samples
Epoch 1/10
40896/40934 [============================>.] - ETA: 0s - loss: 0.4230 - acc: 0.8145

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


40934/40934 [==============================] - 4s 103us/sample - loss: 0.4229 - acc: 0.8146 - val_loss: 0.4763 - val_acc: 0.7362
Epoch 2/10
40934/40934 [==============================] - 4s 88us/sample - loss: 0.3730 - acc: 0.8316 - val_loss: 0.4739 - val_acc: 0.7399
Epoch 3/10
40934/40934 [==============================] - 4s 87us/sample - loss: 0.3670 - acc: 0.8337 - val_loss: 0.4749 - val_acc: 0.7385
Epoch 4/10
40934/40934 [==============================] - 4s 90us/sample - loss: 0.3638 - acc: 0.8354 - val_loss: 0.4744 - val_acc: 0.7392
Epoch 5/10
40934/40934 [==============================] - 4s 89us/sample - loss: 0.3607 - acc: 0.8353 - val_loss: 0.4572 - val_acc: 0.7388
Epoch 6/10
40934/40934 [==============================] - 4s 99us/sample - loss: 0.3578 - acc: 0.8359 - val_loss: 0.4513 - val_acc: 0.7401
Epoch 7/10
40934/40934 [==============================] - 4s 93us/sample - loss: 0.3534 - acc: 0.8365 - val_loss: 0.4480 - val_acc: 0.7391
Epoch 8/10
40934/40934 [=============

In [24]:

y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_names = ['is_Attack', 'is_Normal']
print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[5.9234351e-09 1.0000000e+00]
 [7.4232429e-01 2.5767574e-01]
 [2.5523683e-01 7.4476320e-01]
 ...
 [5.9234466e-09 1.0000000e+00]
 [2.3449093e-01 7.6550913e-01]
 [5.9234466e-09 1.0000000e+00]]
[1 0 1 ... 1 1 1]
              precision    recall  f1-score   support

   is_Attack     0.7471    0.0800    0.1445     12963
   is_Normal     0.7400    0.9898    0.8469     34302

    accuracy                         0.7403     47265
   macro avg     0.7436    0.5349    0.4957     47265
weighted avg     0.7420    0.7403    0.6542     47265

[[ 1037 11926]
 [  351 33951]]
